In [1]:
data_source = ['math-amc', 'math-aime', 'gpqa', 'mip-insufficient', 'mip-question', 'bbh', 'bbh_boolean_expressions', 'bbh_causal_judgement', 'bbh_date_understanding', 'bbh_disambiguation_qa', 'bbh_logical_deduction_five_objects', 'bbh_movie_recommendation', 'bbh_navigate', 'bbh_object_counting', 'bbh_snarks', 'underthink-bench', ]

In [1]:
import os

os.environ['HF_HOME'] = '/nas-ssd2/joykirat/.cache/huggingface'
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
from transformers import AutoTokenizer, AutoModelForCausalLM
from vllm import LLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B")
# Qwen/Qwen3-4B
# deepseek-ai/DeepSeek-R1-Distill-Llama-8B
# deepseek-ai/DeepSeek-R1-Distill-Qwen-7B
# model = LLM(model="Qwen/Qwen3-4B", gpu_memory_utilization=0.95, tensor_parallel_size=1, task="embed")





INFO 09-12 18:13:03 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 09-12 18:13:04 [__init__.py:239] Automatically detected platform cuda.


In [18]:
import json
data = []
with open('/nas-ssd2/joykirat/code/verl-fork/verl/scripts/train/verlCheckpoint/Summary/qwen4b_dapo_math_10k_context_linear_reward_with_summary_attention_v2/rollout/val_120.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))
        

In [19]:
import numpy as np

accuracy_by_source = {}
for line in data:
    if line['data_source'] not in accuracy_by_source:
        accuracy_by_source[line['data_source']] = []
    accuracy_by_source[line['data_source']].append(line['final_reward'])
    
for source in accuracy_by_source:
    mean_accuracy = np.mean(accuracy_by_source[source])
    print(source, mean_accuracy, len(accuracy_by_source[source]))
    
    

math-amc 0.7710843373493976 83
gpqa 0.4213197969543147 197
math-aime 0.4044943820224719 89


In [20]:
# import numpy as np
# response_length_list = {}
# from tqdm import tqdm
# for line in tqdm(data):
#     if line['data_source'] not in response_length_list:
#         response_length_list[line['data_source']] = []
#     response_length_list[line['data_source']].append(len(tokenizer.encode(line['response'])))
# for source in response_length_list:
#     mean_response_length = np.mean(response_length_list[source])
#     print(source, mean_response_length)
            

In [21]:
import re
from math_verify.errors import TimeoutException
from math_verify.metric import math_metric
from math_verify.parser import ExprExtractionConfig, LatexExtractionConfig

def clean_prediction(model_output: str) -> str:
    """
    Try to normalize model outputs like 'Answer: C. Cotton' into '\boxed{C}'.
    """

    # If boxed answer is already there, keep it
    boxed_match = re.search(r'\\boxed\{([A-Za-z0-9]+)\}', model_output)
    if boxed_match:
        return f"\\boxed{{{boxed_match.group(1)}}}"

    # Otherwise, try to find a single-letter multiple-choice answer
    choice_match = re.search(r'\b([A-E])\b', model_output)
    if choice_match:
        return f"\\boxed{{{choice_match.group(1)}}}"

    # If nothing found, just return as-is
    return model_output.strip()

def getRawCorrectness(model_output: str, ground_truth: str) -> float:
    verify_func = math_metric(
        gold_extraction_target=(LatexExtractionConfig(),),
        pred_extraction_target=(ExprExtractionConfig(), LatexExtractionConfig()),
    )

    ret_score = 0.0
    ground_truth_boxed = f"\\boxed{{{ground_truth}}}"

    try:
        pred_clean = clean_prediction(model_output)
        ret_score, _ = verify_func([ground_truth_boxed], [pred_clean])
    except Exception:
        pass

    return ret_score

def getCorrectness(model_output: str, ground_truth: str) -> float:
    if '</think>' in model_output:
        model_output = model_output.split('</think>')[-1]

    verify_func = math_metric(
        gold_extraction_target=(LatexExtractionConfig(),),
        pred_extraction_target=(ExprExtractionConfig(), LatexExtractionConfig()),
    )

    ret_score = 0.0
    ground_truth_boxed = f"\\boxed{{{ground_truth}}}"

    try:
        pred_clean = clean_prediction(model_output)
        ret_score, _ = verify_func([ground_truth_boxed], [pred_clean])
    except Exception:
        pass

    return ret_score


In [22]:
import numpy as np
correctness_response_length_list = {}
from tqdm import tqdm
for line in tqdm(data):
    if line['data_source'] not in correctness_response_length_list:
        correctness_response_length_list[line['data_source']] = []
    correctness_response_length_list[line['data_source']].append((getCorrectness(line['response'], line['ground_truth']), len(tokenizer.encode(line['response']))))
    
            
            
            

100%|██████████| 369/369 [00:04<00:00, 87.96it/s] 


In [23]:
def compute_aucoaa(data):
    # data is a list of (correctness, response_length) tuples
    n = len(data)
    tmax = max(length for _, length in data)
    oaa_t_list = []
    for t in range(tmax + 1):
        filtered = [correct for correct, length in data if length < t]
        if filtered:
            oaa_t = sum(filtered) / n  # Denominator is always n, per definition
        else:
            oaa_t = 0.0
        oaa_t_list.append(oaa_t)
    aucoaa = sum(oaa_t_list) / len(oaa_t_list)
    return aucoaa


In [24]:
for key in correctness_response_length_list.keys():
    print("Data Source: ", key)
    print("Accuracy: ", np.mean([correct for correct, _ in correctness_response_length_list[key]]))
    print("Response Length: ", np.mean([length for _, length in correctness_response_length_list[key]]))
    print("Number of Samples: ", len(correctness_response_length_list[key]))
    print("AUC-OAA: ", compute_aucoaa(correctness_response_length_list[key]))
    print("--------------------------------")

Data Source:  math-amc
Accuracy:  0.7710843373493976
Response Length:  4263.963855421687
Number of Samples:  83
AUC-OAA:  0.5256124989910648
--------------------------------
Data Source:  gpqa
Accuracy:  0.4517766497461929
Response Length:  4529.903553299492
Number of Samples:  197
AUC-OAA:  0.29917718888009676
--------------------------------
Data Source:  math-aime
Accuracy:  0.4044943820224719
Response Length:  6912.483146067416
Number of Samples:  89
AUC-OAA:  0.21470100180993137
--------------------------------


In [9]:
line['response']

"<think>\nOkay, let's try to solve this problem. So, Ellina has twelve blocks, two of each of six colors. An arrangement is even if between each pair of the same color, there's an even number of blocks. Like the example given, where for each color, the two blocks have an even count in between. For example, the two B's in the example have some blocks in between. Let's think about how to calculate the probability.\n\nFirst, the total number of possible arrangements is the number of ways to arrange twelve blocks where there are two of each color. That would be 12! divided by (2!^6), since each color has two identical blocks. So total possibilities is 12!/(2!^6). That's the denominator of the probability.\n\nThe numerator is the number of even arrangements. So we need to find how many arrangements meet the even condition. The condition is that for each color, the two blocks have an even number of blocks between them. Let's think about what that means. For two blocks of the same color, the 

In [10]:
len(tokenizer.encode(line['response']))

4407

In [37]:
import json
## read jsonl
with open('your_file.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]


FileNotFoundError: [Errno 2] No such file or directory: 'your_file.jsonl'

In [1]:
from datasets import load_dataset
train_dataset = load_dataset('/nas-ssd2/joykirat/code/verl-fork/baselines/LC-R1/dataset', split='train')

Generating train split:   0%|          | 0/2499 [00:00<?, ? examples/s]

In [2]:
train_dataset

Dataset({
    features: ['problem', 'solution'],
    num_rows: 2499
})

In [3]:
train_dataset[0]

{'problem': 'Let $x$ , $y$ and $z$ all exceed $1$ and let $w$ be a positive number such that $\\log_xw=24$ , $\\log_y w = 40$ and $\\log_{xyz}w=12$ . Find $\\log_zw$ .',
 'solution': '60'}

In [9]:
## load parquet dataset
import pandas as pd
dataset = pd.read_parquet('/nas-ssd2/joykirat/code/verl-fork/verl/scripts/data/dapo-17k/train.parquet')
dataset.head()

,data_source,prompt,ability,reward_model,extra_info,uuid
0,DAPO-Math-17k,"[{'content': 'In triangle $ABC$, $\sin \angle ...",math,"{'ground_truth': '34', 'style': 'rule-lighteva...","{'index': 0, 'split': 'train'}",d2ccd328-e712-4d6b-b909-a7956589db4e
1,DAPO-Math-17k,[{'content': 'Let $ABCD$ be a unit square in t...,math,"{'ground_truth': '113', 'style': 'rule-lightev...","{'index': 1, 'split': 'train'}",5fb8b773-1e5f-48ab-b5ce-8d2ba6ed53cc
2,DAPO-Math-17k,"[{'content': 'Let $a, b, c$ be distinct number...",math,"{'ground_truth': '-3', 'style': 'rule-lighteva...","{'index': 2, 'split': 'train'}",974a3162-fb31-4720-86ab-6a3738ae84e7
3,DAPO-Math-17k,[{'content': 'There are $7$ boxes arranged in ...,math,"{'ground_truth': '3', 'style': 'rule-lighteval...","{'index': 3, 'split': 'train'}",c47003a1-29c7-46ad-b566-977ba9018acb
4,DAPO-Math-17k,[{'content': 'Let $S$ be the set of triples $(...,math,"{'ground_truth': '37', 'style': 'rule-lighteva...","{'index': 4, 'split': 'train'}",f8e041a5-2848-4dfb-99c4-74aa70c8d211


In [10]:
# convert this dataset into a dataset
from datasets import Dataset
dataset = Dataset.from_pandas(dataset)


In [12]:
dataset

Dataset({
    features: ['data_source', 'prompt', 'ability', 'reward_model', 'extra_info', 'uuid'],
    num_rows: 20000
})

In [13]:
dataset = load_dataset('parquet', data_files='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/data/dapo-17k/train.parquet')

Generating train split: 0 examples [00:00, ? examples/s]

In [17]:
### remove some columsn from the dataset
dataset = dataset.remove_columns(['data_source', 'ability', 'extra_info', 'uuid'])

In [21]:
dataset['train'][0]

{'prompt': [{'content': "In triangle $ABC$, $\\sin \\angle A = \\frac{4}{5}$ and $\\angle A < 90^\\circ$. Let $D$ be a point outside triangle $ABC$ such that $\\angle BAD = \\angle DAC$ and $\\angle BDC = 90^\\circ$. Suppose that $AD = 1$ and that $\\frac{BD}{CD} = \\frac{3}{2}$. If $AB + AC$ can be expressed in the form $\\frac{a\\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$. Let's think step by step and output the final answer within \\boxed{}.",
   'role': 'user'}],
 'reward_model': {'ground_truth': '34', 'style': 'rule-lighteval/MATH_v2'}}

In [22]:
dataset = dataset['train']

In [24]:
# create a new column with problem name from prompt
dataset = dataset.add_column('problem', [d['prompt'][0]['content'].replace("Let's think step by step and output the final answer within \\boxed{}.", '').strip() for d in dataset])


In [27]:
dataset = dataset.add_column('solution', [d['reward_model']['ground_truth'] for d in dataset])

In [29]:
dataset = dataset.remove_columns(['prompt', 'reward_model'])

In [30]:
dataset

Dataset({
    features: ['problem', 'solution'],
    num_rows: 20000
})

In [34]:
### save the dataset locally in dataset format '/nas-ssd2/joykirat/code/verl-fork/baselines/LC-R1/dapo-17k'
from datasets import load_dataset, load_from_disk
# dataset.save_to_disk('/nas-ssd2/joykirat/code/verl-fork/baselines/LC-R1/dapo-17k')

In [35]:
# load the dataset from the local dataset format
dataset = load_from_disk('/nas-ssd2/joykirat/code/verl-fork/baselines/LC-R1/dapo-17k')